In [5]:
u.user <- read.csv(file = 'u.user.csv', sep='|', header=T)
u.item <- read.csv(file = 'u.item.csv', sep='|', header=T)
u.data <- read.csv(file = 'u.data.csv', sep='|', header=T)
colMeans.sparse <- function(m) {colSums(m)/colSums(m>0)}
rowMeans.sparse <- function(m) {rowSums(m)/rowSums(m>0)}
means.sparse <- function(v) {sum(v)/sum(v>0)}

In [6]:
library(Matrix)


In [7]:
head(u.data)

user.id,item.id,rating,timestamp
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596
298,474,4,884182806


In [8]:
head(u.item)

movie.id,movie.title,release.date,video.release.date,IMDb.URL,unknown,Action,Adventure,Animation,Children.s,...,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),01-Jan-1995,NA,http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
v.u <- merge(u.data, u.user, by.x='user.id', by.y='id')

In [10]:
head(v.u)

user.id,item.id,rating,timestamp,age,gender,job,zip
1,1,5,874965758,24,M,technician,85711
1,23,4,875072895,24,M,technician,85711
1,223,5,876892918,24,M,technician,85711
1,171,5,889751711,24,M,technician,85711
1,16,5,878543541,24,M,technician,85711
1,73,3,876892774,24,M,technician,85711


In [11]:
q11 <- aggregate(v.u$rating, FUN=mean, by=list(v.u$job))

In [12]:
q11

Group.1,x
administrator,3.635646
artist,3.653380
doctor,3.688889
educator,3.670621
engineer,3.541407
entertainment,3.441050
executive,3.349104
healthcare,2.896220
homemaker,3.301003
lawyer,3.735316


In [13]:
# groub by groupe de 5

q11 <- aggregate(v.u$rating, FUN=mean, by=list(v.u$age))
q11

Group.1,x
7,3.767442
10,3.387097
11,2.925926
13,3.470825
14,3.375000
15,3.264484
16,3.331343
17,3.516165
18,3.696710
19,3.400968


In [14]:
## Cosinus entre un vecteur v et chaque colonne dela matrice m
cosinus.vm <- function(v,m) { n <- sqrt(colSums(m^2)); (v %*% m)/(n * sqrt(sum(v^2))) }

# Trouve les indexes des premieres 'n' valeurs maximales d'une matrice
max.nindex <- function(m, n=5) {
    i <- order(m, decreasing=TRUE)
    return(i[1:n])
}
# Trouve les indexes des premieres 'n' valeurs minimales d'une matrice
min.nindex <- function(m, n=5) {
    i <- order(m)
    return(i[1:n])
}

In [15]:
m.sparse <- sparseMatrix(u.data[,1],u.data[,2],x=u.data[,3])
m <- as.matrix(m.sparse)
m[m==0] <- NA
rownames(m) <- paste('u', 1:nrow(m), sep='')
colnames(m) <- paste('i', 1:ncol(m), sep='')

In [16]:
ST_id <- u.item$movie.id[u.item$movie.title=="Star Trek V: The Final Frontier (1989)"]
ST_id

[1] 450

In [17]:
ST.cos <- cosinus.vm(m.sparse[,ST_id], m.sparse)
closestCos <- max.nindex(as.matrix(ST.cos), 11)[-1]
closestCos <- u.item$movie.title[closestCos]
closestCos

[1] Star Trek: The Motion Picture (1979)         
 [2] Star Trek VI: The Undiscovered Country (1991)
 [3] Star Trek III: The Search for Spock (1984)   
 [4] Star Trek IV: The Voyage Home (1986)         
 [5] Star Trek: The Wrath of Khan (1982)          
 [6] Stargate (1994)                              
 [7] Star Trek: Generations (1994)                
 [8] Die Hard 2 (1990)                            
 [9] Escape from New York (1981)                  
[10] Conan the Barbarian (1981)                   
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [24]:
## retourne la correlation d une matrice creuse
cor.vm <- function(v,m) {
    v_a <-sum(v)/sum(v>0);
    v_i <- colMeans.sparse(m);
    m.center <- t(t(m) - v_i)*(m>0);
    v.center <- (v-v_a)*(v>0);
    n <- colSums(m.center^2);
    (v.center %*% m.center)/sqrt(n * sum(v.center^2))
}
ST.cor <- cor.vm(m.sparse[,ST_id], m.sparse)
closestCor <- max.nindex(as.matrix(ST.cor), 11)[-1]
closestCor <- u.item$movie.title[closestCor]
closestCor

[1] Star Trek: The Motion Picture (1979)      
 [2] Star Trek III: The Search for Spock (1984)
 [3] Robocop 3 (1993)                          
 [4] Just Cause (1995)                         
 [5] Three Musketeers, The (1993)              
 [6] Star Trek: Generations (1994)             
 [7] Money Train (1995)                        
 [8] House Party 3 (1994)                      
 [9] Mr. Magoo (1997)                          
[10] S.F.W. (1994)                             
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [25]:
#essaie
temp <-(((m.sparse[,450] - m.sparse))^2)*((m.sparse[,450]>0- (m.sparse>0)))^2
new.dist <- sqrt(colSums((temp>0)))

In [86]:
voisins <- 21
# centrer selon lutilisateur les notes des films quil a donne

u.mean <-rowMeans.sparse(m.sparse)
m.center <- as.matrix((m.sparse-u.mean)*(m.sparse>0))
m.center[m.center ==0] <- NA
distance.na.450.center <- sqrt(colSums((m.center[,450] - m.center)^2, na.rm=T)) # ignore les valeurs manquantes


votes.communs <- (colSums((m.sparse[,450] * m.sparse) > 0))
distance.na.450.center[votes.communs<20] <- Inf


# prendre films si plus ou = 40 votes en commun avec 450


In [88]:
closest.euc <- min.nindex(distance.na.450.center, voisins)[-1]
closestEuc <- u.item$movie.title[closest.euc]
closestEuc

[1] Wolf (1994)                         Hot Shots! Part Deux (1993)        
 [3] Judge Dredd (1995)                  Addams Family Values (1993)        
 [5] Last Action Hero (1993)             Santa Clause, The (1994)           
 [7] First Knight (1995)                 Cliffhanger (1993)                 
 [9] Alien 3 (1992)                      Species (1995)                     
[11] Three Musketeers, The (1993)        Basic Instinct (1992)              
[13] Jackie Chan's First Strike (1996)   Dirty Dancing (1987)               
[15] So I Married an Axe Murderer (1993) Lawnmower Man, The (1992)          
[17] Mortal Kombat (1995)                Crying Game, The (1992)            
[19] Young Guns (1988)                   Brady Bunch Movie, The (1995)      
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [89]:
wcos.st <- cosinus.vm(m.sparse[,ST_id], m.sparse[,closest.euc])
wcos.st

1 x 20 Matrix of class "dgeMatrix"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]      [,7]
[1,] 0.2382002 0.2383731 0.3401054 0.3142436 0.3264016 0.2483806 0.2834764
          [,8]      [,9]     [,10]     [,11]     [,12]    [,13]     [,14]
[1,] 0.3308025 0.3070313 0.3150624 0.2805219 0.3050582 0.200634 0.2429202
         [,15]     [,16]    [,17]    [,18]     [,19]     [,20]
[1,] 0.2036197 0.2434685 0.263777 0.208557 0.3027186 0.2784911

In [93]:
# les 20 films les plus communs
idx<- closest.euc
# on transpose m
m.sparse.t <- t(m.sparse)

v.item <- m.sparse.t[450,]
v_1 <-means.sparse(v.item);
# return means for each item
v_i <- apply(m.sparse.t[idx,], 1, FUN=means.sparse)
# I_ij <- apply(c(m.sparse.t[idx,],v_i,v_1), 1, FUN=means.sparse)

x<- as.matrix((m.sparse.t[idx,]))
new_x <-lapply(seq_len(ncol(x)), function(i) x[,i])
new_y <-as.vector(lapply(seq_len(ncol(x)), function(i) v_i))
new_w <-as.vector(lapply(seq_len(ncol(x)), function(i) as.matrix(wcos.st)))

try <- function(x,y,w) {x.0<-x>0;x.temp <- (x-y)*x.0;num <-  w %*%x.temp;denum <- sum(abs(t(w))*x.0);num/denum}


c_prod <- mapply(try ,new_x, new_y, new_w)

prediction <- v_1+c_prod


In [97]:
x.real <-m.sparse.t[450,]
x.real.0 <- x.real>0
sum((prediction-x.real)^2*(x.real.0), na.rm=TRUE)
sqrt(sum((prediction-x.real)^2*(x.real.0), na.rm=TRUE)/sum(x.real.0))

[1] 44.71858

[1] 0.8425074

[1]  943 1682

In [230]:
# creation de lutilisateur
new.user <- rep(0, 1682)
new.user.na <- new.user
new.user.na[new.user.na == 0] <- NA 

In [231]:
# film de star wars
new.user[172] <- 1
new.user[181] <- 1
# film de star treck
indices.star.trek <- grep("trek", as.character(u.item$movie.title), ignore.case=T)
new.user[indices.star.trek] <- 5

In [237]:
voisins <- 20
# centrer selon lutilisateur les notes des films quil a donne

u.mean <-rowMeans.sparse(m.sparse)
m.center <- as.matrix((m.sparse-u.mean)*(m.sparse>0))
m.center[m.center ==0] <- NA
distance.na.450.center <- sqrt(rowSums((new.user.na - m.center)^2, na.rm=T)) # ignore les valeurs manquantes


votes.communs <- (colSums((new.user * m.sparse) > 0))
distance.na.450.center[votes.communs<5] <- Inf

In [238]:
closest.euc <- min.nindex(distance.na.450.center, voisins)
closestEuc <- u.user$id[closest.euc]
closestEuc

[1]   1   4  22  56 181 222 258 286 288 313   2   3   5   6   7   8   9  10  11
[20]  12

In [241]:
length(new.user)
dim(m.sparse)

[1] 1682

[1]  943 1682

In [242]:
# le poids des voisins
wcos.st <- cosinus.vm(new.user, t(m.sparse[closest.euc,]))
wcos.st
# moyenne des votes du nouvel usager
v.new.user <-mean(new.user, na.rm=TRUE)

1 x 20 Matrix of class "dgeMatrix"
          [,1] [,2]      [,3]      [,4]       [,5]      [,6] [,7]       [,8]
[1,] 0.1282891    0 0.1763873 0.1984466 0.03934767 0.1737428    0 0.02880007
           [,9]      [,10] [,11]      [,12]     [,13] [,14]     [,15]     [,16]
[1,] 0.02000504 0.09671505     0 0.01257377 0.2174722     0 0.1084249 0.2386677
     [,17]      [,18]    [,19]      [,20]
[1,]     0 0.02442959 0.145794 0.05302938

In [246]:
idx<- closest.euc
# on transpose m

v.item <- new.user
v_1 <-means.sparse(v.item);
# return means for each item
v_i <- apply(m.sparse[idx,], 1, FUN=means.sparse)
# I_ij <- apply(c(m.sparse.t[idx,],v_i,v_1), 1, FUN=means.sparse)

x<- as.matrix((m.sparse[idx,]))
new_x <-lapply(seq_len(ncol(x)), function(i) x[,i])
new_y <-as.vector(lapply(seq_len(ncol(x)), function(i) v_i))
new_w <-as.vector(lapply(seq_len(ncol(x)), function(i) as.matrix(wcos.st)))

try <- function(x,y,w) {x.0<-x>0;x.temp <- (x-y)*x.0;num <-  w %*%x.temp;denum <- sum(abs(t(w))*x.0);num/denum}


c_prod <- mapply(try ,new_x, new_y, new_w)

prediction <- v_1+c_prod
recomendation <- max.nindex(prediction, 11)


In [248]:
recomendation
closestEuc <- u.item$movie.title[recomendation]
closestEuc

[1] 974 320 347 750  48 390 346   9 458 834 846

[1] Eye for an Eye (1996)                                      
 [2] Paradise Lost: The Child Murders at Robin Hood Hills (1996)
 [3] Wag the Dog (1997)                                         
 [4] Amistad (1997)                                             
 [5] Hoop Dreams (1994)                                         
 [6] Fear of a Black Hat (1993)                                 
 [7] Jackie Brown (1997)                                        
 [8] Dead Man Walking (1995)                                    
 [9] Nixon (1995)                                               
[10] Halloween: The Curse of Michael Myers (1995)               
[11] To Gillian on Her 37th Birthday (1996)                     
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [155]:
dim(m.center)

[1]  943 1682

In [173]:
sum(!is.na(new.user - t(m.center)))

[1] 2299